<a href="https://colab.research.google.com/github/PaulRosenthal/Coastal-Florida-Climate-Changes/blob/main/Weather_Data_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook can be used to retrieve National Weather Service forecast information for a specified area. The forecast is then exported to a CSV file for use.

In [ ]:
import requests, json, csv
from datetime import datetime

api_url = 'https://api.weather.gov/'

# The National Weather Service provides instructions for retrieving the gridpoint information
# for a given area at https://www.weather.gov/documentation/services-web-api.
api_endpoint = 'gridpoints/TAE/24,71/forecast'
request_url = f'{api_url}{api_endpoint}'

# Request the forecast and save it to the response variable.
response = requests.get(request_url)

# Process the response. If status is code is a value other than 200, it is not valid.
if response.status_code == 200:
  # Assign a JSON variable to the response.
  data = response.json()

  # Create a CSV file.
  weather_data_file = open('weather_data.csv', 'w')
  weather_data_file_writer = csv.writer(weather_data_file)

  # Write the first row of the file containing the time and date the forecast was updated.
  forecast_created_time = datetime.fromisoformat(data['properties']['updated'])
  weather_data_file_writer.writerow(['Forecast Generated At: ', forecast_created_time, ' ', ' ', ' ', ' '])

  # Write the header row for the CSV file.
  weather_data_file_writer.writerow(['Time Period Number', 'Start Time', 'End Time', 'Day or Night', 'Days Out From Forecast Generation', 'Forecasted Temperature'])

  # Extract the forecast details into a variable.
  forecast = data['properties']['periods']

  # Parse each time period within the forecast and write the information to the CSV file.
  for time_period in forecast:
    # Time of day determined from isDaytime flag included in the forecast.
    day_or_night = "Night"
    if time_period['isDaytime']:
      day_or_night = "Day"
    
    # Write time period's forecast data to the CSV file.
    time_period_start = datetime.fromisoformat(time_period['startTime'])
    weather_data_file_writer.writerow([time_period['number'], time_period['startTime'], time_period['endTime'], day_or_night, time_period_start - forecast_created_time, time_period['temperature']])

  # Close the CSV file.
  weather_data_file.close()

else:
  # For troubleshooting, the failure message includes the status code and request URL.
  print(f'The forecast could not be retrieved. The response status code received was: {response.status_code}')
  print('The request URL was: ', request_url)